In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
y2015 = pd.read_csv(
    'https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,Bookkeeper/Accounting,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,297xx,SC,33.18,0.0,Jun-1991,0.0,33.0,2.0,11.0,2.0,19108.0,29.6%,19.0,w,13668.88,13668.88,4519.68,4519.68,...,0.0,0.0,1.0,6.0,2848.0,263953.0,17.6,0.0,0.0,131.0,294.0,11.0,6.0,2.0,11.0,76.0,11.0,76.0,2.0,6.0,9.0,6.0,8.0,6.0,9.0,11.0,9.0,11.0,0.0,0.0,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,tech,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,299xx,SC,22.44,0.0,Jun-1996,0.0,NaN,NaN,7.0,0.0,7722.0,59.4%,9.0,w,6635.69,6635.69,3572.97,3572.97,...,0.0,0.0,2.0,2.0,7912.0,4778.0,61.8,0.0,0.0,234.0,91.0,9.0,9.0,0.0,11.0,NaN,9.0,NaN,0.0,3.0,3.0,3.0,3.0,5.0,4.0,4.0,3.0,7.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,Sales Manager,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,226xx,VA,26.02,0.0,Dec-2001,1.0,NaN,NaN,9.0,0.0,20862.0,54.3%,19.0,w,0.00,0.00,26224.23,26224.23,...,1.0,0.0,1.0,2.0,33976.0,17538.0,54.3,0.0,0.0,142.0,168.0,13.0,13.0,3.0,13.0,NaN,0.0,NaN,0.0,3.0,3.0,5.0,6.0,7.0,5.0,9.0,3.0,9.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,Senior Manager,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,275xx,NC,21.60,0.0,May-1984,0.0,42.0,NaN,16.0,0.0,51507.0,64.5%,24.0,w,19263.77,19263.77,10271.36,10271.36,...,0.0,1.0,0.0,1.0,13819.0,16623.0,59.9,0.0,0.0,82.0,379.0,19.0,19.0,2.0,19.0,48.0,NaN,42.0,0.0,7.0,9.0,9.0,11.0,4.0,13.0,18.0,9.0,16.0,0.0,0.0,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,Program Coordinator,8 years,RENT,55000.0,Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,462xx,IN,25.49,0.0,Mar-2005,4.0,NaN,30.0,18.0,1.0,9568.0,46%,19.0,w,0.00,0.00,9190.49,9190.49,...,2.0,0.0,5.0,17.0,1051.0,1375.0,45.0,0.0,0.0,129.0,95.0,0.0,0.0,0.0,8.0,NaN,0.0,NaN,0.0,2.0,17.0,2.0,2.0,2.0,17.0,17.0,13.0,18.0,NaN,0.0,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [4]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [7]:
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [12]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421088,36221446.0,38932858.0,11575.0,11575.0,11575.0,36 months,15.59,404.61,D,10+ years,RENT,25400.0,Verified,Jan-2015,Fully Paid,n,credit_card,Credit card refinancing,27.08,0.0,0.0,NaN,NaN,9.0,0.0,18266.0,18.0,f,0.00,0.00,13976.74,13976.74,11575.00,2401.74,0.0,0.0,0.0,Sep-2016,6281.14,NaN,...,NaN,NaN,NaN,1.0,2200.0,9234.0,66.4,0.0,0.0,113.0,184.0,48.0,8.0,0.0,48.0,NaN,14.0,NaN,0.0,7.0,7.0,7.0,10.0,5.0,8.0,13.0,7.0,9.0,0.0,0.0,0.0,1.0,100.0,42.9,0.0,0.0,29866.0,19798.0,27500.0,1766.0
421089,36330911.0,39042282.0,12000.0,12000.0,12000.0,36 months,11.99,398.52,B,< 1 year,MORTGAGE,63000.0,Verified,Jan-2015,Charged Off,n,credit_card,Credit card refinancing,23.69,1.0,0.0,9.0,NaN,13.0,0.0,21110.0,61.0,f,0.00,0.00,5164.77,5164.77,3847.36,1317.41,0.0,0.0,0.0,Feb-2016,398.52,NaN,...,NaN,NaN,NaN,11.0,18231.0,4776.0,69.0,0.0,0.0,121.0,233.0,1.0,1.0,1.0,1.0,NaN,14.0,NaN,1.0,4.0,8.0,4.0,5.0,40.0,8.0,20.0,8.0,13.0,0.0,0.0,1.0,3.0,98.3,50.0,0.0,0.0,325484.0,112990.0,15400.0,98125.0
421090,36371250.0,39102635.0,10000.0,10000.0,10000.0,36 months,11.99,332.10,B,8 years,RENT,31000.0,Verified,Jan-2015,Current,n,debt_consolidation,Debt consolidation,28.69,0.0,0.0,NaN,77.0,9.0,1.0,14037.0,15.0,f,3737.83,3737.83,7962.24,7962.24,6262.17,1700.07,0.0,0.0,0.0,Jan-2017,332.10,Feb-2017,...,NaN,NaN,NaN,2.0,3159.0,448.0,95.1,0.0,0.0,122.0,115.0,17.0,10.0,0.0,90.0,NaN,9.0,NaN,0.0,3.0,6.0,3.0,4.0,7.0,7.0,8.0,6.0,9.0,0.0,0.0,0.0,1.0,100.0,100.0,0.0,0.0,32950.0,25274.0,9200.0,15850.0
421091,36441262.0,39152692.0,24000.0,24000.0,24000.0,36 months,11.99,797.03,B,10+ years,MORTGAGE,79000.0,Verified,Jan-2015,Current,n,home_improvement,Home improvement,3.90,0.0,1.0,26.0,NaN,5.0,0.0,8621.0,23.0,f,8971.06,8971.06,19096.75,19096.75,15028.94,4067.81,0.0,0.0,0.0,Jan-2017,797.03,Feb-2017,...,NaN,NaN,NaN,3.0,35071.0,976.0,89.2,0.0,0.0,NaN,488.0,3.0,3.0,4.0,3.0,29.0,3.0,29.0,10.0,2.0,3.0,3.0,14.0,0.0,4.0,17.0,3.0,5.0,0.0,0.0,0.0,2.0,56.5,100.0,0.0,0.0,152650.0,8621.0,9000.0,0.0
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,debt_consolidation,Debt consolidation,30.90,0.0,0.0,NaN,84.0,9.0,1.0,11031.0,22.0,w,0.00,0.00,5034.02,5034.02,2527.70,2506.32,0.0,0.0,0.0,May-2016,316.07,NaN,...,NaN,NaN,NaN,7.0,3798.0,2554.0,75.9,0.0,0.0,134.0,108.0,6.0,6.0,0.0,17.0,NaN,10.0,NaN,0.0,4.0,6.0,4.0,4.0,12.0,7.0,10.0,6.0,9.0,0.0,0.0,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0


In [11]:
y2015 = y2015[:-2]
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421088,36221446.0,38932858.0,11575.0,11575.0,11575.0,36 months,15.59,404.61,D,10+ years,RENT,25400.0,Verified,Jan-2015,Fully Paid,n,credit_card,Credit card refinancing,27.08,0.0,0.0,NaN,NaN,9.0,0.0,18266.0,18.0,f,0.00,0.00,13976.74,13976.74,11575.00,2401.74,0.0,0.0,0.0,Sep-2016,6281.14,NaN,...,NaN,NaN,NaN,1.0,2200.0,9234.0,66.4,0.0,0.0,113.0,184.0,48.0,8.0,0.0,48.0,NaN,14.0,NaN,0.0,7.0,7.0,7.0,10.0,5.0,8.0,13.0,7.0,9.0,0.0,0.0,0.0,1.0,100.0,42.9,0.0,0.0,29866.0,19798.0,27500.0,1766.0
421089,36330911.0,39042282.0,12000.0,12000.0,12000.0,36 months,11.99,398.52,B,< 1 year,MORTGAGE,63000.0,Verified,Jan-2015,Charged Off,n,credit_card,Credit card refinancing,23.69,1.0,0.0,9.0,NaN,13.0,0.0,21110.0,61.0,f,0.00,0.00,5164.77,5164.77,3847.36,1317.41,0.0,0.0,0.0,Feb-2016,398.52,NaN,...,NaN,NaN,NaN,11.0,18231.0,4776.0,69.0,0.0,0.0,121.0,233.0,1.0,1.0,1.0,1.0,NaN,14.0,NaN,1.0,4.0,8.0,4.0,5.0,40.0,8.0,20.0,8.0,13.0,0.0,0.0,1.0,3.0,98.3,50.0,0.0,0.0,325484.0,112990.0,15400.0,98125.0
421090,36371250.0,39102635.0,10000.0,10000.0,10000.0,36 months,11.99,332.10,B,8 years,RENT,31000.0,Verified,Jan-2015,Current,n,debt_consolidation,Debt consolidation,28.69,0.0,0.0,NaN,77.0,9.0,1.0,14037.0,15.0,f,3737.83,3737.83,7962.24,7962.24,6262.17,1700.07,0.0,0.0,0.0,Jan-2017,332.10,Feb-2017,...,NaN,NaN,NaN,2.0,3159.0,448.0,95.1,0.0,0.0,122.0,115.0,17.0,10.0,0.0,90.0,NaN,9.0,NaN,0.0,3.0,6.0,3.0,4.0,7.0,7.0,8.0,6.0,9.0,0.0,0.0,0.0,1.0,100.0,100.0,0.0,0.0,32950.0,25274.0,9200.0,15850.0
421091,36441262.0,39152692.0,24000.0,24000.0,24000.0,36 months,11.99,797.03,B,10+ years,MORTGAGE,79000.0,Verified,Jan-2015,Current,n,home_improvement,Home improvement,3.90,0.0,1.0,26.0,NaN,5.0,0.0,8621.0,23.0,f,8971.06,8971.06,19096.75,19096.75,15028.94,4067.81,0.0,0.0,0.0,Jan-2017,797.03,Feb-2017,...,NaN,NaN,NaN,3.0,35071.0,976.0,89.2,0.0,0.0,NaN,488.0,3.0,3.0,4.0,3.0,29.0,3.0,29.0,10.0,2.0,3.0,3.0,14.0,0.0,4.0,17.0,3.0,5.0,0.0,0.0,0.0,2.0,56.5,100.0,0.0,0.0,152650.0,8621.0,9000.0,0.0
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,debt_consolidation,Debt consolidation,30.90,0.0,0.0,NaN,84.0,9.0,1.0,11031.0,22.0,w,0.00,0.00,5034.02,5034.02,2527.70,2506.32,0.0,0.0,0.0,May-2016,316.07,NaN,...,NaN,NaN,NaN,7.0,3798.0,2554.0,75.9,0.0,0.0,134.0,108.0,6.0,6.0,0.0,17.0,NaN,10.0,NaN,0.0,4.0,6.0,4.0,4.0,12.0,7.0,10.0,6.0,9.0,0.0,0.0,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0


In [13]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,...,last_pymnt_d_Nov-2016,last_pymnt_d_Oct-2015,last_pymnt_d_Oct-2016,last_pymnt_d_Sep-2015,last_pymnt_d_Sep-2016,next_pymnt_d_Feb-2017,next_pymnt_d_Jan-2017,next_pymnt_d_Jul-2016,next_pymnt_d_Mar-2017,last_credit_pull_d_Apr-2015,last_credit_pull_d_Apr-2016,last_credit_pull_d_Aug-2015,last_credit_pull_d_Aug-2016,last_credit_pull_d_Dec-2014,last_credit_pull_d_Dec-2015,last_credit_pull_d_Dec-2016,last_credit_pull_d_Feb-2015,last_credit_pull_d_Feb-2016,last_credit_pull_d_Jan-2015,last_credit_pull_d_Jan-2016,last_credit_pull_d_Jan-2017,last_credit_pull_d_Jul-2015,last_credit_pull_d_Jul-2016,last_credit_pull_d_Jun-2015,last_credit_pull_d_Jun-2016,last_credit_pull_d_Mar-2015,last_credit_pull_d_Mar-2016,last_credit_pull_d_May-2015,last_credit_pull_d_May-2016,last_credit_pull_d_Nov-2015,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,0.0,33.0,2.0,11.0,2.0,19108.0,19.0,13668.88,13668.88,4519.68,4519.68,2331.12,2188.56,0.0,0.0,0.0,379.39,0.0,33.0,1.0,NaN,NaN,0.0,0.0,31329.0,1.0,2.0,1.0,2.0,6.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,0.0,NaN,NaN,7.0,0.0,7722.0,9.0,6635.69,6635.69,3572.97,3572.97,2964.31,608.66,0.0,0.0,0.0,298.58,0.0,NaN,1.0,NaN,NaN,0.0,0.0,55387.0,0.0,3.0,0.0,0.0,47.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,1.0,NaN,NaN,9.0,0.0,20862.0,19.0,0.00,0.00,26224.23,26224.23,25000.00,1224.23,0.0,0.0,0.0,20807.39,0.0,NaN,1.0,NaN,NaN,0.0,0.0,305781.0,0.0,3.0,0.0,1.0,13.0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,0.0,42.0,NaN,16.0,0.0,51507.0,24.0,19263.77,19263.77,10271.36,10271.36,8736.23,1535.13,0.0,0.0,0.0,858.05,0.0,NaN,1.0,NaN,NaN,0.0,0.0,221110.0,0.0,2.0,0.0,0.0,27.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,4.0,NaN,30.0,18.0,1.0,9568.0,19.0,0.00,0.00,9190.49,9190.49,8650.00,540.49,0.0,0.0,0.0,8251.42,0.0,NaN,1.0,NaN,NaN,0.0,0.0,18926.0,6.0,1.0,0.0,1.0,19.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421088,36221446.0,38932858.0,11575.0,11575.0,11575.0,15.59,404.61,25400.0,27.08,0.0,0.0,NaN,NaN,9.0,0.0,18266.0,18.0,0.00,0.00,13976.74,13976.74,11575.00,2401.74,0.0,0.0,0.0,6281.14,0.0,NaN,1.0,NaN,NaN,0.0,0.0,19798.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
421089,36330911.0,39042282.0,12000.0,12000.0,12000.0,11.99,398.52,63000.0,23.69,1.0,0.0,9.0,NaN,13.0,0.0,211

In [16]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X=y2015.drop('loan_status',1)
Y=y2015['loan_status']
X=pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc,X,Y,cv=10)

array([0.98185704, 0.98104963, 0.98207077, 0.98192785, 0.97941058,
       0.97779572, 0.96233584, 0.98102543, 0.98100169, 0.98093044])

In [19]:
np.unique(y2015['loan_status'])

array(['Charged Off', 'Current', 'Default', 'Fully Paid',
       'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)'],
      dtype=object)

In [22]:
target_dict = {'Charged Off':0, 'Default':1, 'Late (31-120 days)':2, 'Late (16-30 days)':3, 'In Grace Period':4, 'Current':5, 'Fully Paid':6}
y2015['loan_status'] = [target_dict[i] for i in y2015['loan_status']]

In [23]:
if 'id' in y2015.columns:
    y2015.drop(columns=['id'], inplace=True)
tar_corr = y2015.corr().sort_values(by=['loan_status'], ascending=False)['loan_status']
tar_corr

loan_status                1.000000
total_rec_prncp            0.373933
last_pymnt_amnt            0.324154
total_pymnt_inv            0.312964
total_pymnt                0.312957
                             ...   
total_rec_late_fee        -0.132540
int_rate                  -0.200794
recoveries                -0.410818
collection_recovery_fee   -0.412490
policy_code                     NaN
Name: loan_status, Length: 87, dtype: float64

In [24]:
features = tar_corr.index[tar_corr > 0.2].to_list()
features.extend(tar_corr.index[tar_corr < 0.15].to_list())
features

['loan_status',
 'total_rec_prncp',
 'last_pymnt_amnt',
 'total_pymnt_inv',
 'total_pymnt',
 'annual_inc_joint',
 'bc_open_to_buy',
 'tot_hi_cred_lim',
 'avg_cur_bal',
 'mort_acc',
 'total_bc_limit',
 'tot_cur_bal',
 'out_prncp_inv',
 'out_prncp',
 'mo_sin_old_rev_tl_op',
 'total_rev_hi_lim',
 'mths_since_recent_inq',
 'mo_sin_rcnt_tl',
 'mo_sin_rcnt_rev_tl_op',
 'member_id',
 'mths_since_recent_bc',
 'annual_inc',
 'mths_since_rcnt_il',
 'mo_sin_old_il_acct',
 'max_bal_bc',
 'total_cu_tl',
 'num_bc_tl',
 'revol_bal',
 'mths_since_last_delinq',
 'mths_since_recent_revol_delinq',
 'num_rev_accts',
 'total_acc',
 'mths_since_recent_bc_dlq',
 'tot_coll_amt',
 'mths_since_last_major_derog',
 'total_bal_ex_mort',
 'pct_tl_nvr_dlq',
 'num_bc_sats',
 'total_il_high_credit_limit',
 'delinq_amnt',
 'num_tl_30dpd',
 'chargeoff_within_12_mths',
 'num_tl_120dpd_2m',
 'num_accts_ever_120_pd',
 'acc_now_delinq',
 'total_bal_il',
 'num_tl_90g_dpd_24m',
 'tax_liens',
 'pub_rec_bankruptcies',
 'pub_rec

In [28]:
temp_df = y2015[features]
temp_df.corr().sort_values(by=['loan_status'],ascending=False)
rfc = ensemble.RandomForestClassifier()
X=temp_df.drop('loan_status',1)
Y=temp_df['loan_status']
X=pd.get_dummies(X)
X=X.dropna(axis=1)

score = cross_val_score(rfc,X,Y,cv=10)
score

array([0.96426027, 0.96739492, 0.96411779, 0.96188463, 0.9610772 ,
       0.9610772 , 0.96100596, 0.96083973, 0.96067349, 0.96043601])